In [2]:
import os

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from pymongo import MongoClient

load_dotenv()

/Users/sookhyun/Documents/personal/langchain-rag-poc/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [3]:
client = MongoClient(os.environ["DEV_MONGO_CONNECTION_STRING"])

DB_NAME = "lab_dev"
COLLECTION_NAME = "langchain_embedding"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "test_vector_store_index"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
    embedding=embeddings,
)

# Also can be done manually through console
# vector_store.create_vector_search_index(dimensions=1536)

In [5]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10},
)

llm = ChatOpenAI(model="gpt-4o")

message = """
You are an AI concierge. Answer the question using the provided context only.
You must answer in Korean with "informal language".

User Name: {username}
Question: {question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("human", message)
])

In [8]:
def format_input(inputs):
    return {
        "question": inputs["question"],
        "username": inputs["username"],
        "context": retriever.invoke(inputs["question"])
    }

rag_chain = RunnableLambda(format_input) | prompt | llm

In [18]:
response = rag_chain.invoke({"username": "__b1ob", "question": "단풍을 보러 가고싶어."})
print(response.content)

단풍 보러 갈 거면 담양의 관방제림 추천해! 가을 풍경이 정말 멋지다고 해. 나무가 울창하고 주황색과 붉은색 단풍이 가득해서 걷기에도 좋고 자연을 느끼기에 완벽한 곳이야. 조용한 강가 길도 있어서 산책하기에 더할 나위 없이 좋을 거야. 한 번 가봐!


In [19]:
response = rag_chain.invoke({"username": "__b1ob", "question": "눈오는데 갈만한 공간 없을까?"})
print(response.content)

눈 올 때 갈만한 곳으로 마이오운가든 추천해! 영등포구에 있는 아늑한 카페인데, 따뜻한 분위기에 커피와 디저트가 맛있어. 눈 오는 날 창밖을 보며 커피 한 잔 즐기기 좋은 곳이야.


In [20]:
response = rag_chain.invoke({"username": "__b1ob", "question": "여자친구와 한국적인 공간을 방문하고싶어."})
print(response.content)

여자친구랑 한국적인 공간을 찾고 있다면, 서울 동작구에 있는 한옥 카페 R1을 추천해. 이곳은 전통 한국 건축물인 한옥에 현대적인 도시 감각을 더한 멋진 카페야. 335년 된 은행나무가 있어 분위기도 좋고, 자연광이 잘 드는 창문 덕분에 아늑한 느낌이 들어. 커피 마시면서 사진 찍기도 좋고, 조용히 책 읽으면서 힐링하기에도 완벽한 장소야. 꼭 가봐!


In [21]:
response = rag_chain.invoke({"username": "__b1ob", "question": "한옥카페 R1은 주차가 가능해?"})
print(response.content)

한옥카페 R1에 대한 주차 정보는 제공되지 않았어. 다른 정보를 참고하거나 직접 문의해보는 게 좋을 것 같아!


In [22]:
response = rag_chain.invoke({"username": "__b1ob", "question": "서울에 여자친구랑 가기 좋은 주차 가능한 공간이 있을까?"})
print(response.content)

서울에서 여자친구랑 가기 좋은 주차 가능한 공간으로는 "The Plaza Seoul, Autograph Collection" 호텔이 있어. 이 호텔은 서울의 중심부인 중구에 위치하고 있고, 주변에 광화문, 명동, 덕수궁 같은 주요 명소들이 있어서 문화적인 탐방을 하기에 좋을 것 같아. 호텔은 현대적이고 세련된 디자인을 자랑하며, 도심의 활기찬 풍경을 즐길 수 있는 숙박 경험을 제공해.
